### Imports

In [1]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import mnist

### Inicialização de Pesos

In [2]:
def he(cLayer_size,size,pLayer_size):
    #parametros:
    #cLayer = tamanho da camada atual
    #pLayer = tamanho da camada anterior
    #w=np.random.randn(cLayer_size,size)*np.sqrt(2/pLayer_size)
    std_dev = np.sqrt(2.0 / (cLayer_size + size))
    return std_dev*np.random.randn(cLayer_size, size)
    #return w

### Normalização de batch

In [3]:
def normbatch(batch):

    for b in range(batch.shape[0]):
            me = np.mean(batch[b],axis=(0,1,2))
            std = np.std(batch[b],axis=(0,1,2))
            batch[b] = (batch[b] - me)/std
    return batch            


### Função para Cálculo de Perdas

In [4]:
def softmax(x):
    z = []
    soma = np.sum(np.exp(x))
    z = (np.exp(x)/soma)
    
    return np.asarray_chkfinite(z)

### Separação em batchs

In [5]:
def batchsep(conj,bsize):
    #conj2 = np.zeros((conj.shape[0]//bsize,size,size))
    #conj2 = np.asarray(conj2)
    conj2 = []
    #np.random.shuffle(conj)
    #print(conj.shape[0])
    for i in range(0,conj.shape[0],bsize):
       # np.append(conj2, conj[i:i+bsize,:,:], axis = 0)
       conj2.append(conj[i:i+bsize])
       # print(conj[i:i+bsize,:,:], i)
    return normbatch(np.asarray(conj2,dtype=float))    

def batchsep2(conj,bsize):
    #conj2 = np.zeros((conj.shape[0]//bsize,size,size))
    #conj2 = np.asarray(conj2)
    conj2 = []
    #np.random.shuffle(conj)
    #print(conj.shape[0])
    for i in range(0,conj.shape[0],bsize):
       # np.append(conj2, conj[i:i+bsize,:,:], axis = 0)
       conj2.append(conj[i:i+bsize])
       # print(conj[i:i+bsize,:,:], i)
    return np.asarray(conj2)  



### Pooling

In [6]:
def pool(img, size, band=0):
    #parametros 
    #img = imagem de entrada
    #size = tamanho da matriz de pool
    rec = []
    img = np.asarray(img)
    raio = size//2
    sh = len(img.shape)
    img2 = []
    k =0;
    l = 0;

    if(sh>2):
      #  print('HHHHHHH')
        img2 = np.zeros((img.shape[0],int(img.shape[1]//2),int(img.shape[2]//2)))
        for i in range(raio,img.shape[1]-raio+1,2):
            for j in range(raio,img.shape[2]-raio+1,2):
                for s in range(img.shape[0]):
                    rec = img[s,i-raio : i+raio,j-raio : j+raio]
                    img2[s,k, l] = np.amax(rec)
                l+=1
            k+=1
            l =0    
    else:
      #  print('Help')
        img2 = np.zeros((int(img.shape[0]//2),int(img.shape[1]//2)))
        for i in range(raio,img.shape[0]-raio+1,2):
            for j in range(raio,img.shape[1]-raio+1,2):
                rec = img[i-raio : i+raio,j-raio : j+raio]
                img2[k, l] = np.amax(rec)
                l+=1
            k+=1
            l =0    

    return img2            

### Padding

In [7]:
def padd(img, raio):
    sh = len(img.shape)
    if (sh > 2):
        for i in range(raio):
            img =  np.insert(img,0,0,axis = 3)
            img =  np.insert(img,img.shape[3],0,axis =3)
            img =  np.insert(img,0,0,axis = 2)
            img =  np.insert(img,img.shape[2],0,axis = 2)
    else:
        for i in range(raio):
            img =  np.insert(img,0,0,axis = 0)
            img =  np.insert(img,img.shape[0],0,axis = 0)
            img =  np.insert(img,0,0,axis = 1)
            img =  np.insert(img,img.shape[1],0,axis = 1)
    return img           

### Função de ativação

In [8]:
def actv(func, z, b=0):
    z =np.asarray(z)
    sh =len(z.shape)
    if(sh > 2):

        if(func == 1):
            for k in range(z.shape[2]):
                for i in range(z.shape[0]):
                    for j in range(z.shape[1]):
                        z[i,j,k] = max(0, (z[i,j,k]+b))
            
        else:
            for k in range(z.shape[2]):
                for i in range(z.shape[0]):
                    for j in range(z.shape[1]):
                        z[i,j,k] =255*((2/(1 +math.exp(-2*z[i,j,k])))-1 +b)
    else:
        if(func == 1):
            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    z[i,j] = max(0, (z[i,j]+b))
            
        else:
            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    z[i,j] =255*((2/(1 +math.exp(-2*z[i,j])))-1 +b )          
    return z 

def actv2(func, z, b=0):
    z =np.asarray(z)
    sh =len(z.shape)
    if(sh > 2):

        if(func == 1):
            for k in range(z.shape[0]):
                for i in range(z.shape[1]):
                    for j in range(z.shape[2]):
                        #print(z[k,i,j])
                        z[k,i,j] = max(0, (z[k,i,j]+b))
            
        else:
            for k in range(z.shape[0]):
                for i in range(z.shape[1]):
                    for j in range(z.shape[2]):
                        z[k,i,j] =255*((2/(1 +math.exp(-2*z[k,i,j])))-1 +b)
    else:
        if(func == 1):
            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    z[i,j] = max(0, (z[i,j]+b))
            
        else:
            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    z[i,j] =255*((2/(1 +math.exp(-2*z[i,j])))-1 +b )          
    return z

def actv3(func,z,b):
    for i in range(z.shape[0]):
        z[i] = max(0,z[i]+b)
    return z    

### Convolução

In [9]:
def conv(img, filt,band=0 ,stride=1, pad=False ,sel= False):
    #tam_img = len(img)
    sfilt = len(filt.shape)

    if sfilt > 2:
        raio = filt.shape[1]//2
        tam_filt = filt.shape[1]
    else:
        raio = filt.shape[0]//2
        tam_filt = filt.shape[0]    
    
    sh =len(img.shape)
    img2 = []
    img3 = []
    rec = []
    rec2 = []
    rec3 = []
    if(sh > 2):
        
        if(sel == False):

            if(pad == False):
                #img2 = np.zeros((int(((img.shape[0]-tam_filt)/stride)+1),int(((img.shape[1]-tam_filt)/stride)+1)),dtype = object)
                img3 = np.zeros((int(((img.shape[1]-tam_filt)/stride)+1),int(((img.shape[2]-tam_filt)/stride)+1)),dtype = object)
                if (sfilt > 2):
                
                    for i in range(0, img.shape[0],stride):
                        for j in range(raio, img.shape[1]-raio,stride):
                            for k in range(raio,img.shape[2]-raio):
                                rec = img[i,j-raio : j+raio+1,k-raio : k+raio+1]
                                
                                 
                                img3[j-raio][k-raio] = np.sum(rec * filt[i])
                else:
                    for i in range(0, img.shape[0],stride):
                        for j in range(raio, img.shape[1]-raio,stride):
                            for k in range(raio,img.shape[2]-raio):
                                rec = img[i,j-raio : j+raio+1,k-raio:k+raio]
                                
                                 
                                img3[i-raio][j-raio] = np.sum(rec * filt)
                                 

            else:
               img2 = padd(img,raio)
               #print(img2.shape)
               img3 = np.zeros((int((len(img)-tam_filt +2)/stride)+1,int((img.shape[1]-tam_filt +2)/stride)+1))
              # print(img3.shape)
               if (sfilt >2):
                   
                    for i in range(raio, img.shape[0],stride):
                        for j in range(raio, img.shape[1]-raio,stride):
                            for k in range(0,img.shape[2]-raio):
                                rec = img2[i-raio : i+raio+1,j-raio : j+raio+1,k]
                                img3[i-raio][j-raio] += np.sum(rec * filt[k])
               else:
                    for i in range(raio, img.shape[0],stride):
                        for j in range(raio, img.shape[1]-raio,stride):
                            for k in range(0,img.shape[2]-raio):
                                rec = img2[i-raio : i+raio+1,j-raio : j+raio+1,k]
                                img3[i-raio][j-raio] += np.sum(rec * filt)

        else:
            if(pad == False):
                img2 = np.zeros((int(((img.shape[0]-tam_filt)/stride)+1),int(((img.shape[1]-tam_filt)/stride)+1)),dtype = object)
                img3 = np.zeros((int(((img.shape[0]-tam_filt)/stride)+1),int(((img.shape[1]-tam_filt)/stride)+1)),dtype = object)
                for i in range(raio, img.shape[0]-raio,stride):
                    for j in range(raio, img.shape[1]-raio,stride):
                        rec = img[i-raio : i+raio+1,j-raio : j+raio+1,band]
                        img3[i-raio][j-raio] = np.sum(rec * filt)

            else:
               img2 = padd(img,raio)
              # print(img2.shape)
               img3 = np.zeros((int((len(img)-tam_filt +2)/stride)+1,int((img.shape[1]-tam_filt +2)/stride)+1))
              # print(img3.shape)
               for i in range(raio, img.shape[0]-raio,stride):
                    for j in range(raio, img.shape[1]-raio,stride):
                        rec = img2[i-raio : i+raio+1,j-raio : j+raio+1,band]
                        img3[i-raio][j-raio] = np.sum(rec * filt)

    else:
        if(pad == False):
          #  img2 = np.zeros((int(((img.shape[0]-tam_filt)/stride)+1),int(((img.shape[1]-tam_filt)/stride)+1)),dtype = object)
            img3 = np.zeros((int(((img.shape[0]-tam_filt)/stride)+1),int(((img.shape[1]-tam_filt)/stride)+1)),dtype = object)
            for i in range(raio, img.shape[0]-raio,stride):
                    for j in range(raio, img.shape[1]-raio,stride):
                         rec = img[i-raio : i+raio+1,j-raio : j+raio+1][0]
                         #print(rec.shape)
                        # print(rec.shape[0], rec.shape[1])
                         img3[i-raio][j-raio] = np.sum(rec * filt)
        else:
           img2 = padd(img,raio)
           img3 = np.zeros((int((len(img)-tam_filt +2)/stride)+1,int((img.shape[1]-tam_filt +2)/stride)+1))
           for i in range(raio, img.shape[0]-raio,stride):
                  for j in range(raio, img.shape[1]-raio,stride):
                         rec = img2[i-raio : i+raio+1,j-raio : j+raio+1]
                         img3[i-raio][j-raio] = np.sum(rec * filt)
    return img3

### Camada Convolucional

In [10]:
def convLayer(img,C,act, filters,stride=1, pad=False,sel=False):
    x = []
    if(C > 1):

        fmap =[conv(img,filters[k],k,stride,pad,sel) for k in range(C)]
       
        fmap = actv2(act,fmap,0)

    else:
        if (len(filters.shape) > 2):
            fmap =[conv(img,filters[k],stride,pad = False) for k in range(filters.shape[0])]
            #for i in range(0, C):
            fmap = actv2(act,fmap,0)
        else:
            fmap = conv(img,filters,stride,pad = False)
            fmap = actv(act,fmap,0)
        
    return fmap

### Arquitetura da Lenet

In [11]:
def lenet(img,imgl):

    #treino---------------------------------------------------------------    
    learning_rate = 0.0001
    h = 0.0001
    momentum =0.1
    #padding
    img =padd(img,2)
    
    #variáveis para armazenar as entradas das convoluções e
    #poolings e camadas completamente conectadas
    x1 = []
    x2 = []
    x3 = []
    x4 = []
    x5 = []
    x6 = []
    x7 = []
    x8 = []
    x9 = []

    #saídas das backs

    #inicialização dos pesos
    #pes1 = [he(5,5,6) for i in range(6)]
    pes1 = [he(5,5,6) for i in range(1)]
    pes1 = np.asarray(pes1)
    pes2 = []
    #pes2 = [[he(5,5,6) for _ in range(6)] for _ in range(16)]
    pes2 = [[he(5,5,1) for _ in range(1)] for _ in range(2)]
    pes2 = np.asarray(pes2)
    #pes3 = [[he(5,5,16) for _ in range(16)] for _ in range(120)]
    pes3 = [[he(5,5,2) for _ in range(2)] for _ in range(1)]
    pes3 = np.asarray(pes3)
    #pes4 = he(120,1,1)
    pes5 = he(84,1,1)
    pes6 = he(10,1,84)
    #derivada dos pesos
    dpes1 = np.zeros((pes1.shape[0],pes1.shape[1],pes1.shape[2]))
    dpes2 = np.zeros((pes2.shape[0],pes2.shape[1],pes2.shape[2],pes2.shape[3]))
    dpes3 = np.zeros((pes3.shape[0],pes3.shape[1],pes3.shape[2],pes3.shape[3]))
    #dpes4 = np.zeros(len(pes4))
    #dpes4.shape = (len(pes4),1)
    dpes5 = np.zeros(len(pes5))
    dpes5.shape = (len(pes5),1)
    dpes6 = np.zeros(len(pes6))
    dpes6.shape = (len(pes6),1)
    filtros = []
    #filtros.append([pes1,pes2,pes3,pes4,pes5,pes6])
    filtros.append([pes1,pes2,pes3,pes5,pes6])
    dfiltros = []
    #dfiltros.append([dpes1,dpes2,dpes3,dpes4,dpes5,dpes6])
    dfiltros.append([dpes1,dpes2,dpes3,dpes5,dpes6])
    filtros = np.asarray(filtros)
    dfiltros = np.asarray(dfiltros)
    #print(dfiltros[0,0].shape,filtros[0,0].shape,sep = '\n')
    #for xyz in range(6):
    #    print(filtros[0,xyz].shape,sep = '\n')
    #return 0
    #print(filtros[0,1])
    #return 0
    l = 0
    ct = 0
    erros = [0]*32
    erroslayer = [0]*32
    erroslayernovo = [0]*32
    saidas= [0]*32
    
    for imgt in range(img.shape[0]):
        if (imgt == 32):
            for i in erros:
                print(i)
            return
        for imgb in img[imgt]:
            x1 = imgb
            #imgb = convLayer(imgb,6,1,pes1)
            #print(imgb)
            imgb = convLayer(imgb,1,1,filtros[0,0])
            x2 =imgb
            imgb= pool(imgb, 2) #maxpool
            
           # print(imgb)
           # print('XD')
            x3 = imgb
            #imgb = convLayer(imgb,16,1,pes2)
            imgb = convLayer(imgb,2,1,filtros[0,1])
            x4 = imgb
            imgb = pool(imgb,2) #maxpool
            x5 = imgb
            imgb = convLayer(imgb,1,1,filtros[0,2])
            
            #totalmente conectada
            x6 = imgb
            imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
            x7 = imgb
            imgb = actv3(1,imgb,0)
            imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
            #imgb = actv3(1,imgb,0)
            #x8 = imgb
            #imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,5]])
            #imgb = actv3(1,imgb,0)
            x9 = imgb
            
            #print(x1,x2,x3,x4,x5,x6,x7,x8,x9,sep = '\n')
            imgb = softmax(imgb)
            #print(max(imgb))
            try:
                igual = list(imgb).index(max(imgb))
            except:
                erroslayer[l] = 0
            saidas[l] = imgb
            if( igual != imgl[imgt][l]):
                try:
                    erroslayer[l] = -np.log10(imgb[imgl[imgt][l]])
                except:
                    erroslayer[l] = 0
                erros[imgt] = erros[imgt] + 1
            l = l+1    
        l = 0
        print(erros[imgt])
        for k in range(len(filtros[0])):
            if(k > 0):
                print(np.mean(erroslayernovo))
           # print(len(filtros[0,k].flat))
            for s in range(len(filtros[0,k].flat)):
                if (s > 0):
                    filtros[0,k].flat[s-1] = filtros[0,k].flat[s-1]-h
                filtros[0,k].flat[s] =  filtros[0,k].flat[s]+h
                #if (k > 2):
                #    print(filtros[0,k].flat[s])
                for imgb in img[imgt]:
                   
                    imgb = convLayer(imgb,1,1,filtros[0,0])                   
                    imgb= pool(imgb, 2) #maxpool                    
                  
                    imgb = convLayer(imgb,2,1,filtros[0,1])                    
                    imgb = pool(imgb,2) #maxpool
                    
                    imgb = convLayer(imgb,1,1,filtros[0,2])                    
                    #totalmente conectada                   
                    imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
                    imgb = actv3(1,imgb,0)
                    
                    imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
                    #imgb = actv3(1,imgb,0)
                   # imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,5]])
                   # imgb = actv3(1,imgb,0)
                    
                  
                    imgb = softmax(imgb)
                    try:
                        igual = list(imgb).index(max(imgb))
                    except:
                        erroslayernovo[l] = 0
                        print(imgb)
                        return 0
                        
                    
                    saidas[l] = imgb
                    if( igual != imgl[imgt][l]):
                        try:
                            erroslayernovo[l] = -np.log10(imgb[imgl[imgt][l]])
                        except:
                            erroslayernovo[l] =0
                            #print('Sigh')
                    l = l+1    
                l = 0
                #print(len(dfiltros[0,k].flat),k,s,sep = ' ')
                dfiltros[0,k].flat[s] = (np.mean(erroslayernovo) -np.mean(erroslayer) )/h
            for k in range(5):
                filtros[0,k] = filtros[0,k] - learning_rate*dfiltros[0,k]
            

        '''for imgb in img[imgt]:
            dpes1 = dpes1 * momentum +(1-momentum)*erroslayer[l]
            pes6 = pes6 - learning_rate*dpes1
            pes5 = pes5 - learning_rate*dpes1
            pes4 = pes4 - learning_rate*dpes1
            pes3 = pes3 - learning_rate*dpes1
            pes2 = pes2 - learning_rate*dpes1
            #print(pes1)
            pes1 = pes1 - learning_rate*dpes1
            l = l+1
        l = 0'''    

        '''for imgb in img[imgt]:
            #1
            for k in range(len(saidas[l])):
                derivative = 1
                oSignals[k] = derivative * (saida[l][k] - imgb[imgl[imgt][l]])
            #2 start modifications
            for j in range(len(saidas[l])):
                for k in range(len(x8)):
                    hoGrads[j] += oSignals[j] * x8[k]
            #4 
            for j in range(len(pes6)):
                sum = 0.0
                for k in range(len(saidas[l])):
                    sum+= oSignals[k] * pes6[j]
                    derivative = 1
                hSignals[j] = derivative * sum'''
                    

                            
            #vou ter que colocar fors de acordo com minha qtde de camadas
            #no #4 o selfhoWeights vai ser o peso da camada anterior
            #vou ter q fazer vários hSignals
            # no #5 provavelmente será usado meu último hsignal            



        #backwards  
        '''(dx9,dpes6) = fc_backward(x9,x8,pes6)
        (dx8,dpes5) = fc_backward(dx9,x7,pes5)
        (dx7,dpes4) = fc_backward(dx8,x6,pes4)
        print(pes3.shape)
        
        (dx6,dpes3) = conv_backward(x6,x5,pes3)
        dx5 = pool_backward(dx6,x4)
        (dx4,dpes2) = conv_backward(dx5,x3,pes2)
        dx3 = pool_backward(dx4,x2)
        (dx2, dpes1) = conv_backward(dx3,x1,pes1
        dx9 = erroslayer * pes6
        dx8 = erroslayer * pes5
        dx7 = erroslayer * pes4
        #for xyz in range(pes3.shape[0]):
        #    for poke in range(pes3.shape[1]):
        dx6 = erroslayer * pes3 
        dx4 = erroslayer * pes2
        dx2 = erroslayer * pes1

        #atualizando os pesos
        pes6 = pes6 - learning_rate*np.random.random_sample()*dx9
        pes5 = pes5 - learning_rate*np.random.random_sample()*dx8
        pes4 = pes4 - learning_rate*np.random.random_sample()*dx7
        pes3 = pes3 - learning_rate*np.random.random_sample()*dx6
        pes2 = pes2 - learning_rate*np.random.random_sample()*dx4
        pes1 = pes1 - learning_rate*np.random.random_sample()*dx2
        '''

        



    



    #validação-----------------------------------------------------------

In [12]:
    img_train = mnist.train_images()
    img_train_labels = mnist.train_labels()
    #print(img_train.shape)
    #test_images = mnist.test_images()
    #test_labels = mnist.test_labels()
    img_train = batchsep(img_train,32)
    img_train_labels = batchsep2(img_train_labels,32)
    #print(img_train.shape)
    #print(np.array_equal(img_train[1][0],img_train[1][1]))
    #padd(img_train,4)

In [26]:
#lenet(img_train,img_train_labels)
img = img_train 
imgl = img_train_labels
#treino---------------------------------------------------------------    
learning_rate = 0.0001
h = 0.0001
momentum =0.1
help = 0
#padding
img =padd(img,2)

#variáveis para armazenar as entradas das convoluções e
#poolings e camadas completamente conectadas
x1 = []
x2 = []
x3 = []
x4 = []
x5 = []
x6 = []
x7 = []
x8 = []
x9 = []

#saídas das backs

#inicialização dos pesos

pes1 = [he(5,5,6) for i in range(1)]
pes1 = np.asarray(pes1)
pes2 = []

pes2 = [[he(5,5,1) for _ in range(1)] for _ in range(2)]
pes2 = np.asarray(pes2)

pes3 = [[he(5,5,2) for _ in range(2)] for _ in range(1)]
pes3 = np.asarray(pes3)

pes5 = he(84,1,1)
pes6 = he(10,1,84)

dpes1 = np.zeros((pes1.shape[0],pes1.shape[1],pes1.shape[2]))
dpes2 = np.zeros((pes2.shape[0],pes2.shape[1],pes2.shape[2],pes2.shape[3]))
dpes3 = np.zeros((pes3.shape[0],pes3.shape[1],pes3.shape[2],pes3.shape[3]))
dpes5 = np.zeros(len(pes5))
dpes5.shape = (len(pes5),1)
dpes6 = np.zeros(len(pes6))
dpes6.shape = (len(pes6),1)
filtros = []

filtros.append([pes1,pes2,pes3,pes5,pes6])
dfiltros = []

dfiltros.append([dpes1,dpes2,dpes3,dpes5,dpes6])
filtros = np.asarray(filtros)
dfiltros = np.asarray(dfiltros)




In [27]:
l = 0
ct = 0
erros = [0]*32
erroslayer = [0]*32
erroslayernovo = [0]*32
saidas= [0]*32


In [28]:

for imgt in range(img.shape[0]):
    if (imgt == 32):
        for i in erros:
            print(i)
        
    for imgb in img[imgt]:
        x1 = imgb
        imgb = convLayer(imgb,1,1,filtros[0,0])
        x2 =imgb
        imgb= pool(imgb, 2) #maxpool

        x3 = imgb
       
        imgb = convLayer(imgb,2,1,filtros[0,1])
        x4 = imgb
        imgb = pool(imgb,2) #maxpool
        x5 = imgb
        imgb = convLayer(imgb,1,1,filtros[0,2])
        
        #totalmente conectada
        x6 = imgb
        imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
        x7 = imgb
        imgb = actv3(1,imgb,0)
        imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
        
        x9 = imgb
        
        #print(x1,x2,x3,x4,x5,x6,x7,x8,x9,sep = '\n')
        
        while(help == 0):
            try:
                imgb = softmax(imgb)
                igual = list(imgb).index(max(imgb))
                if( igual != imgl[imgt][l]):
                    if(not(np.isfinite(-np.log(imgb[imgl[imgt][l]])))):
                               raise ValueError
                    erros[imgt] = erros[imgt] + 1
                erroslayer[l] = -np.log(imgb[imgl[imgt][l]])    
                help =1
            except:
                print('SIGH')
                pes1 = [he(5,5,6) for i in range(1)]
                pes1 = np.asarray(pes1)
                pes2 = []
                pes2 = [[he(5,5,1) for _ in range(1)] for _ in range(2)]
                pes2 = np.asarray(pes2)
                pes3 = [[he(5,5,2) for _ in range(2)] for _ in range(1)]
                pes3 = np.asarray(pes3)
                pes5 = he(84,1,1)
                pes6 = he(10,1,84)
                filtros[0,0] = pes1
                filtros[0,1] = pes2
                filtros[0,2] = pes3
                filtros[0,3] = pes5
                filtros[0,4] = pes6
                imgb = [1,1,1,1,1,1,1,1,1,1]
                #imgb = x1
                #imgb = convLayer(imgb,1,1,filtros[0,0])
                #imgb = pool(imgb,2)
                #imgb = convLayer(imgb,2,1,filtros[0,1])
                #imgb = pool(imgb,2)
                #imgb = convLayer(imgb,1,1,filtros[0,2])
                #imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
                #imgb = actv3(1,imgb,0)
                #imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
                
        help = 0
        print(imgb)
        
        l = l+1    
    l = 0
    print(erros[imgt])
    for k in range(len(filtros[0])-2):
        if(k > 0):
            print(np.mean(erroslayernovo))
       # print(len(filtros[0,k].flat))
        for s in range(len(filtros[0,k].flat)):
            if (s > 0):
                filtros[0,k].flat[s-1] -=-h
            filtros[0,k].flat[s] +=h
            #if (k > 2):
            #    print(filtros[0,k].flat[s])
            for imgb in img[imgt]:
                x1 = imgb
                
                imgb = convLayer(imgb,1,1,filtros[0,0])                   
                imgb= pool(imgb, 2) #maxpool                    
              
                imgb = convLayer(imgb,2,1,filtros[0,1])                    
                imgb = pool(imgb,2) #maxpool
                
                imgb = convLayer(imgb,1,1,filtros[0,2])                    
                #totalmente conectada                   
                imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
                imgb = actv3(1,imgb,0)
                
                imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
               
              
                
                while(help == 0):    
                    try:
                        imgb = softmax(imgb)
                        igual = list(imgb).index(max(imgb))
                        saidas[l] = imgb
                        if( igual != imgl[imgt][l]):
                            if(not(np.isfinite(-np.log(imgb[imgl[imgt][l]])))):
                               raise ValueError
                        erroslayernovo[l] = -np.log(imgb[imgl[imgt][l]])
                        help =1
                    except:
                        print('WHY?')
                        pes1 = [he(5,5,6) for i in range(1)]
                        pes1 = np.asarray(pes1)
                        pes2 = []
                        pes2 = [[he(5,5,1) for _ in range(1)] for _ in range(2)]
                        pes2 = np.asarray(pes2)
                        pes3 = [[he(5,5,2) for _ in range(2)] for _ in range(1)]
                        pes3 = np.asarray(pes3)
                        pes5 = he(84,1,1)
                        pes6 = he(10,1,84)
                        filtros[0,0] = pes1
                        filtros[0,1] = pes2
                        filtros[0,2] = pes3
                        filtros[0,3] = pes5
                        filtros[0,4] = pes6
                        imgb = [1,1,1,1,1,1,1,1,1,1]
                        #imgb = x1
                        #imgb = convLayer(imgb,1,1,filtros[0,0])
                        #imgb = pool(imgb,2)
                        #imgb = convLayer(imgb,2,1,filtros[0,1])
                        #imgb = pool(imgb,2)
                        #imgb = convLayer(imgb,1,1,filtros[0,2])
                        #imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,3]])
                        #imgb = actv3(1,imgb,0)
                        #imgb = np.asarray([np.sum(imgb * i) for i in filtros[0,4]])
                
                help =0    
                l = l+1    
            l = 0
            #print(len(dfiltros[0,k].flat),k,s,sep = ' ')
            dfiltros[0,k].flat[s] = (np.mean(erroslayernovo) -np.mean(erroslayer) )/h
        print('XD')    
        for k in range(3):
            filtros[0,k] -= learning_rate*dfiltros[0,k]

[0.08387133 0.08885631 0.1312767  0.40237347 0.08329209 0.03773442
 0.10562796 0.00919689 0.01944118 0.03832965]
[2.32979534e-07 4.08818586e-07 1.82957210e-05 9.99978642e-01
 2.17773257e-07 9.74904456e-11 2.20225013e-06 1.04178386e-16
 1.52711991e-13 1.13543580e-10]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[2.26189010e-06 3.65036393e-06 9.27791098e-05 9.99883862e-01
 2.13560634e-06 3.01261796e-09 1.53045817e-05 2.49172313e-14
 1.23412582e-11 3.42997870e-09]
[1.31223616e-14 4.25737867e-14 1.21410802e-10 1.00000000e+00
 1.13935821e-14 1.11484263e-21 1.44479720e-12 3.54569562e-34
 1.50053305e-27 1.53380204e-21]
[2.52294682e-15 8.69771331e-15 3.73897561e-11 1.00000000e+00
 2.17466091e-15 9.25468698e-23 3.54021147e-13 6.67058705e-36
 6.20210608e-29 1.29438923e-22]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.

KeyboardInterrupt: 